In [ ]:
!pip install transformers[sentencepiece] datasets &> /dev/null
!pip install sacrebleu sentencepiece &> /dev/null
!pip install huggingface_hub &> /dev/null
!pip install datasets transformers[sentencepiece] sacrebleu &> /dev/null
!pip install telepot &> /dev/null
!pip install evaluate &> /dev/null

In [ ]:
import re
import time
import random
import string
import pathlib
import datasets
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from transformers import AdamWeightDecay
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import TextVectorization
from transformers import TFT5EncoderModel, T5Tokenizer
from transformers.keras_callbacks import PushToHubCallback
from transformers.keras_callbacks import KerasMetricCallback
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AutoTokenizer,GPT2Tokenizer, BertTokenizer, TFBertModel

In [ ]:
# Detect hardware
try:
    tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu_resolver = None
    gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
    tf.config.experimental_connect_to_cluster(tpu_resolver)
    tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
    strategy = tf.distribute.experimental.TPUStrategy(tpu_resolver)
    print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.106.44.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.106.44.26:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.106.44.26:8470']
Number of accelerators:  8


# **Dataset**

In [ ]:
!rm ita-eng.zip &> /dev/null
!rm rm -rf dataset &> /dev/null
!wget "https://www.manythings.org/anki/ita-eng.zip" &> /dev/null
!unzip "ita-eng.zip" -d "dataset" &> /dev/null

text_file = "dataset/ita.txt"

In [ ]:
sequence_length = 90


def split_set(dataset: tf.data.Dataset,
              tr: float = 0.8,
              val: float = 0.1,
              ts: float = 0.1,
              shuffle: bool = True) -> (tf.data.Dataset, tf.data.Dataset, tf.data.Dataset):
    if tr+val+ts != 1:
        raise ValueError("Train, validation and test partition not allowed with such splits")

    dataset_size = dataset.cardinality().numpy()
    if shuffle:
        dataset = dataset.shuffle(dataset_size)

    tr_size = int(tr * dataset_size)
    val_size = int(val * dataset_size)

    tr_set = dataset.take(tr_size)
    val_set = dataset.skip(tr_size).take(val_size)
    ts_set = dataset.skip(tr_size).skip(val_size)
    return tr_set, val_set, ts_set


def make_batches(dataset_src_dst: tf.data.Dataset, batch_size: int):
    return dataset_src_dst.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


def create_dataset(name: str, preprocessed:bool):
    with open(name, encoding="UTF-8") as datafile:
        src_set = list()
        dst_set = list()
        for sentence in datafile:
            sentence = sentence.split("\t")
            src_set.append(sentence[0])
            if preprocessed:
                dst_set.append(sentence[1].split("\n")[0])
            else:
                dst_set.append(sentence[1])

    return src_set, dst_set



def format_dataset(src, trg):
    return ({"encoder_inputs": src, "decoder_inputs": trg[:, :-1]}, trg[:, 1:])

def make_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.prefetch(tf.data.experimental.AUTOTUNE).cache()


# **Tokenizer**


In [ ]:
from transformers import BertTokenizerFast


source_src = "bert-base-uncased" 
target_src = "dbmdz/bert-base-italian-cased" 

tokenizer_src = BertTokenizerFast.from_pretrained(source_src)
tokenizer_trg = BertTokenizerFast.from_pretrained(target_src)

v_size_src = tokenizer_src.vocab_size
v_size_trg = tokenizer_trg.vocab_size

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [ ]:
def tokenize(entire_set):

  source_set, target_set = entire_set
  tokens_source = tokenizer_src(source_set, truncation=True, padding="max_length",
                              return_tensors="tf", max_length=sequence_length).data["input_ids"]
  tokens_source = tf.cast(tokens_source, dtype=tf.int32)                            
  tokens_target = tokenizer_trg(target_set, add_special_tokens=True,
                              truncation=True, padding="max_length",
                              return_tensors="tf", max_length=sequence_length + 1).data["input_ids"]
  tokens_target = tf.cast(tokens_target, dtype=tf.int32)
  return tokens_source, tokens_target

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(tokenize(create_dataset("dataset/ita.txt", False)))

In [ ]:
tr_set, val_set, ts_set = split_set(dataset, 0.9, 0.05, 0.05)  

# **Encoder-Decoder Model**

In [ ]:
dropout_rate = 0.2

In [ ]:
with strategy.scope():
  encoder = TFBertModel.from_pretrained(source_src)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
class EncoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, dropout=0.1, **kwargs) -> None:
        super(EncoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads, layers_size, dropout=dropout)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dropout(dropout), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        if mask is not None:  
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        else:
            print("Mask not built")
            assert False
        
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class EncoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 max_length: int,
                 v_size_src: int,
                 dropout: float = 0.1) -> None:
        super(EncoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length, v_size_src, layers_size)
        self.enc_layers = [EncoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, mask=None) -> tf.Tensor:
        src_embeddings = self.pos_embedding(inputs)
        enc_out = self.dropout(src_embeddings)
        for i in range(self.num_layers):
            enc_out = self.enc_layers[i](enc_out)

        return enc_out  # (batch_size, input_seq_len, layers_size)


class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
      super(PositionalEmbedding, self).__init__(**kwargs)
      self.token_embeddings = layers.Embedding(
          input_dim= vocab_size, output_dim=embed_dim
      )
      self.position_embeddings = layers.Embedding(
          input_dim=sequence_length, output_dim=embed_dim
      )
      self.sequence_length = sequence_length
      self.vocab_size = vocab_size
      self.embed_dim = embed_dim

  def call(self, inputs):
      length = tf.shape(inputs)[-1] 
      positions = tf.range(start=0, limit=length, delta=1)
      embedded_tokens = self.token_embeddings(inputs)
      embedded_positions = self.position_embeddings(positions)
      return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
      return tf.math.not_equal(inputs, 0)


class DecoderLayer(layers.Layer):

    def __init__(self, layers_size: int, dense_size: int, num_heads: int, dropout=0.1, **kwargs) -> None:
        super(DecoderLayer, self).__init__(**kwargs)
        
        self.layers_size = layers_size
        self.dense_size = dense_size
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads, layers_size, dropout=dropout)
        self.attention_2 = layers.MultiHeadAttention(num_heads, layers_size, dropout=dropout)
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_size, activation="relu"), layers.Dropout(dropout), layers.Dense(layers_size)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, encoder_outputs: tf.Tensor, mask=None) -> tf.Tensor:
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)
        

class DecoderTransformer(layers.Layer):

    def __init__(self,
                 num_layers: int,
                 layers_size: int,
                 dense_size: int,
                 num_heads: int,
                 max_length: int,
                 v_size_dst: int,
                 dropout=0.1) -> None:
        super(DecoderTransformer, self).__init__()

        self.layers_size = layers_size
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(max_length, v_size_dst, layers_size)
        self.dec_layers = [DecoderLayer(layers_size, dense_size, num_heads) for _ in range(num_layers)]
        self.dropout = layers.Dropout(dropout)
        self.supports_masking = True

    def call(self, inputs: tf.Tensor, enc_output: tf.Tensor, mask=None) -> tf.Tensor:
        dst_embeddings = self.pos_embedding(inputs)
        dec_output = self.dropout(dst_embeddings)
        for i in range(self.num_layers):
            dec_output = self.dec_layers[i](dec_output, enc_output)

        return dec_output

In [ ]:
sequence_length = 90 


def create_model(layers_size: int, num_layers: int, dense_size: int, num_heads: int, max_length: int, encoder=None) -> tf.keras.Model:
    # Encoder
    encoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="encoder_inputs")
    if encoder is not None: 
        outputs = encoder(encoder_inputs)
        encoder_outputs = outputs.last_hidden_state
        layers_size = encoder_outputs.shape[-1]  # the size of the encoder and decoder layers must be the same
    else:
        encoder_outputs = EncoderTransformer(num_layers, layers_size, dense_size, num_heads, max_length, v_size_src)(encoder_inputs)

    # Decoder
    decoder_inputs = tf.keras.Input(shape=(None,), dtype="int32", name="decoder_inputs")
    encoded_seq_inputs = tf.keras.Input(shape=(None, layers_size), name="decoder_state_inputs")
    decoder_outputs = DecoderTransformer(num_layers, layers_size, dense_size, num_heads, max_length, v_size_trg)(decoder_inputs, encoded_seq_inputs)
    decoder_outputs = layers.Dense(v_size_trg, activation="softmax")(decoder_outputs)
    decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

    # Final model
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    transformer = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")
    return transformer

In [ ]:
with strategy.scope():
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath='./bert2bert.h5',
      monitor = 'val_loss',
      verbose = 1,
      save_weights_only = True,
      mode = 'auto',
      save_freq = 'epoch',
      overwrite = True)

In [ ]:
source_src=  "dbmdz/bert-base-italian-cased" 
target_src = "dbmdz/bert-base-italian-cased" 

tokenizer_source1 = BertTokenizer.from_pretrained(source_src)
tokenizer_target1 = BertTokenizer.from_pretrained(target_src)

def metric(target, output):
    import evaluate
    predictions = [output]
    references = [target]
    bleu = evaluate.load("bleu")
    try:
      results = bleu.compute(predictions=predictions, references=references)["bleu"]
    except:
      return 0
    return results

def clean(word):
    clean_word = word.lower()
    clean_word = clean_word.replace(".", "")
    clean_word = clean_word.replace("\"", "")
    clean_word = clean_word.replace(",", "")
    clean_word = clean_word.replace("’", "'")
    clean_word = clean_word.replace("[start]", "")
    clean_word = clean_word.replace("[end]", "")
    clean_word = clean_word.strip()
    return clean_word


def decode_sequence(input_sentence, tokenizer_source, tokenizer_target, transformer):
    #tokenized_input_sentence=input_sentence
    tokenized_input_sentence = tokenizer_source(input_sentence, return_tensors='tf', add_special_tokens=True, max_length = sequence_length, padding='max_length', truncation=True).data["input_ids"]
    decoded_sentence = "[CLS]"
    list_tokens=[decoded_sentence]
    for i in range(sequence_length):

        decoded_sentence = tokenizer_target.convert_tokens_to_string(list_tokens)
        tokenized_target_sentence = tokenizer_target(decoded_sentence, return_tensors='tf', add_special_tokens=False, max_length = sequence_length, padding='max_length').data['input_ids']
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = tokenizer_target.ids_to_tokens[sampled_token_index] #spa_index_lookup[sampled_token_index]
        
        #decoded_sentence += sampled_token
       
        if sampled_token == "[SEP]":
          decoded_sentence = tokenizer_target.convert_tokens_to_string(list_tokens[1:])
          break
        list_tokens.append(sampled_token)
    
    return list_tokens, decoded_sentence


def calculate_metric_on_testset(modello):
  result = 0
  count = 0
  for item in test_pairs[:num_of_test]:
      input_eng = item[0]   #eng
      target = item[1]  #ita
      # print("target: ", target)
      output = decode_sequence(input_eng, tokenizer_source1, tokenizer_target1, modello)
      item_result = metric(clean(target[1]),clean(output[1]))
      result+=item_result
      count+=1

  final_result = result/count
  return final_result

In [ ]:
text_file = "./dataset/ita.txt"
num_of_tests = 100

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, ita, _ = line.split("\t")
    ita = "[start] " + ita + " [end]"
    text_pairs.append((eng, ita))

import random
random.seed(39081)
random.shuffle(text_pairs)

num_train_samples = int(len(text_pairs) * 0.80)
train_pairs = text_pairs[:num_train_samples]
test_pairs = text_pairs[num_train_samples:]

RANDOM_SEARCH = False # if false, the model will be trained using the following hyperparameters:

BATCH_SIZE = 16 * strategy.num_replicas_in_sync
EMBED_DIM = 512
LATENT_DIM = 1024
NUM_HEAD = 6

random_search_trial = 4
epochs = 1
vocab_size = 15000
sequence_length = 90
max_decoded_sentence_length = 90
num_of_test = 30 #Number of sentences in the test set used to calculate the metric

models = []

if not RANDOM_SEARCH:
    random_search_trial = 1

for _ in range(random_search_trial):

  if not RANDOM_SEARCH:
    batch_size = BATCH_SIZE
  else:
    batch_size = random.choice([8,16]) * strategy.num_replicas_in_sync

  train_ds = make_dataset(tr_set, batch_size)
  val_ds = make_dataset(val_set, batch_size)
  test_ds = make_dataset(ts_set, batch_size)

  if not RANDOM_SEARCH:
    embed_dim = EMBED_DIM
    latent_dim = LATENT_DIM
    num_heads = NUM_HEAD
  else:
    embed_dim = random.choice([512])
    latent_dim = random.choice([32,64,128,256,512])
    num_heads = random.choice([6, 8])

  with strategy.scope():
    print(embed_dim, " ", latent_dim, " ", num_heads, " ", batch_size)
    transformer = create_model( embed_dim, 6, latent_dim, num_heads, sequence_length, encoder)
    transformer.summary()
    opt = tf.keras.optimizers.Adam()
    transformer.compile(opt, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
    history = transformer.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])
    metric_result = calculate_metric_on_testset(transformer)
    lista = [str(batch_size), str(embed_dim), str(latent_dim), str(num_heads), 
            str(metric_result), str(epochs)]
    models.append((transformer,metric_result, lista))

# **Translator**

In [ ]:
def decode_sequence(input_sentence, tokenizer_source, tokenizer_target, transformer):
    #tokenized_input_sentence=input_sentence
    tokenized_input_sentence = tokenizer_source(input_sentence, return_tensors='tf', add_special_tokens=True, max_length = sequence_length, padding='max_length', truncation=True).data["input_ids"]
    decoded_sentence = "[CLS]"
    list_tokens=[decoded_sentence]
    for i in range(sequence_length):

        decoded_sentence = tokenizer_target.convert_tokens_to_string(list_tokens)
        tokenized_target_sentence = tokenizer_target(decoded_sentence, return_tensors='tf', add_special_tokens=False, max_length = sequence_length, padding='max_length').data['input_ids']
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = tokenizer_target.ids_to_tokens[sampled_token_index] #spa_index_lookup[sampled_token_index]
        
        #decoded_sentence += sampled_token
       
        if sampled_token == "[SEP]":
          decoded_sentence = tokenizer_target.convert_tokens_to_string(list_tokens[1:])
          break
        list_tokens.append(sampled_token)
    
    return list_tokens, decoded_sentence

In [ ]:
source_src= "dbmdz/bert-base-italian-cased" 
target_src = "dbmdz/bert-base-italian-cased" 

tokenizer_source1 = BertTokenizer.from_pretrained(source_src)
tokenizer_target1 = BertTokenizer.from_pretrained(target_src)

def translation(sequence):
  with strategy.scope():
    tokens, translated = decode_sequence(sequence, tokenizer_source1, tokenizer_target1, transformer)
  return translated

In [ ]:
to_translate = "I went to the restaurant with my friends"

print([to_translate, translation(to_translate)])

In [ ]:
to_translate = "I ate a lion with a friend and it was delicious"

print([to_translate, translation(to_translate)])

# **Evaluator**

In [ ]:
text_file = "./dataset/ita.txt"
num_of_tests = 100

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, ita, _ = line.split("\t")
    ita = "[start] " + ita + " [end]"
    text_pairs.append((eng, ita))

import random
random.seed(39081)
random.shuffle(text_pairs)

num_train_samples = int(len(text_pairs) * 0.80)
train_pairs = text_pairs[:num_train_samples]
test_pairs = text_pairs[num_train_samples:]

def the_metric(target, output):
    import evaluate
    predictions = [output]
    references = [target]
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=predictions, references=references)["bleu"]
    return results

def clean(word):
    clean_word = word.lower()
    clean_word = clean_word.replace(".", "")
    clean_word = clean_word.replace("\"", "")
    clean_word = clean_word.replace(",", "")
    clean_word = clean_word.replace("’", "'")
    clean_word = clean_word.replace("[start]", "")
    clean_word = clean_word.replace("[end]", "")
    clean_word = clean_word.strip()
    return clean_word


def evaluate_model(translate_function):
    result = 0
    count = 0
    for i, item in enumerate(test_pairs[0:num_of_tests]):
        input_eng = clean(item[0])  # eng
        target = clean(item[1])  # ita
        output = clean(translate_function(input_eng))
        item_result = the_metric(target, output)
        print("=================== TEST #", i, "===================")
        print("🇮🇹 ", target)
        print("🇺🇸 ", input_eng)
        print("Translated: ", output)
        print("BLEU score: ", item_result)
        result += item_result
        count += 1

    final_result = result / count
    return final_result

In [ ]:
result = evaluate_model(translation)
print(result)